In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def load_json(file):
    with open(file) as fp:
        data = json.load(fp)
        return data

In [45]:
def save_data_to_json(data, file):
    with open(file, 'w') as fp:
        json.dump(data, fp)

#### load data

In [3]:
df_train = pd.read_csv('comment_specialty_train.csv')
df_test = pd.read_csv('comment_specialty_test.csv')

In [4]:
df_train

,id,comment,specialty_id
0,19600,Красное высыпание на боку живота,15
1,172287,періодична сипь на руці. по ознаках схожа на к...,15
2,1078909,Номер направлення 6303-9261-6693-4577,8
3,45659,"Удаление двух родинок на спине, скорее всего х...",81
4,4647,нужна консультация,53
...,...,...,...
20955,784405,"Здравствуйте, хочу проконсультироваться на счё...",38
20956,80062,гінеколог,13
20957,13016,"Сломан тазобедренный сустав 10дней назад,одет ...",34
20958,330832,По времени желателен промежуток между 9:00 - 1...,8


In [5]:
import stanza

In [6]:
nlp_ru = stanza.Pipeline(lang="ru", use_gpu=True)

2021-03-20 21:28:01 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2021-03-20 21:28:01 INFO: Use device: gpu
2021-03-20 21:28:01 INFO: Loading: tokenize
2021-03-20 21:28:04 INFO: Loading: pos
2021-03-20 21:28:05 INFO: Loading: lemma
2021-03-20 21:28:05 INFO: Loading: depparse
2021-03-20 21:28:06 INFO: Loading: ner
2021-03-20 21:28:07 INFO: Done loading processors!


In [7]:
stopwords_ru = load_json('stopwords-ru.json')

In [8]:
ACCEPTABLE_UPOS = ['NOUN', 'PROPN', 'VERB', 'ADJ', 'ADV']

In [9]:
def get_important_tokens(text: str):
#     print(text)
    tokens = []
    try:
        sentences = nlp_ru(text).sentences
        for sent in sentences:
            for word in sent.words:
                lemma = word.lemma
                if lemma in stopwords_ru:
                    continue
                if word.upos not in ACCEPTABLE_UPOS:
                    continue
                tokens.append(lemma)
    except:
        pass
    return tokens

In [10]:
get_important_tokens("Красное высыпание на боку живота")

['красный', 'высыпание', 'бок', 'живот']

In [11]:
get_important_tokens("Красное высыпание на 43боку живота")

['красный', 'высыпание', '4кбок', 'живот']

In [12]:
%%time
X_tokens_train, y_ids_train = [], []
for idx, row in df_train.iterrows():
    tokens = get_important_tokens(row.comment)
    if not tokens:
        continue
    X_tokens_train.append(tokens)
    y_ids_train.append(row.specialty_id)

CPU times: user 14min 21s, sys: 2.58 s, total: 14min 24s
Wall time: 14min 24s


In [46]:
save_data_to_json(data={
    'x_tokens': X_tokens_train,
    'y' : y_ids_train
}, file = "train_tokens.json")

In [13]:
%%time
X_tokens_test, y_ids_test = [], []
for idx, row in df_test.iterrows():
    tokens = get_important_tokens(row.comment)
    if not tokens:
        continue
    X_tokens_test.append(tokens)
    y_ids_test.append(row.specialty_id)

CPU times: user 3min 36s, sys: 640 ms, total: 3min 37s
Wall time: 3min 37s


In [47]:
save_data_to_json(data={
    'x_tokens': X_tokens_test,
    'y' : y_ids_test
}, file = "test_tokens.json")

In [49]:
del X_tokens_test

In [50]:
del X_tokens_train

In [14]:
len(X_tokens_train)

20344

In [15]:
len(y_ids_train)

20344

In [16]:
len(df_train)

20960

In [17]:
from collections import Counter

In [18]:
word_counter = Counter()

In [19]:
for tokens in X_tokens_train:
    for token in tokens:
        word_counter[token] += 1

In [20]:
len(word_counter)

15642

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

In [23]:
def dummy(doc):
    return doc

cv = CountVectorizer(
        tokenizer=dummy,
        preprocessor=dummy,
    max_features=512
    )  

In [24]:
X_train = cv.fit_transform(X_tokens_train)

In [25]:
X_test = cv.transform(X_tokens_test)

In [26]:
specialty_to_group = load_json("group_specialty_dict.json")

In [27]:
specialty_to_group['47'] = 46

In [28]:
specialty_to_group['33'] = 33

In [29]:
y_group_train = [specialty_to_group[str(yi)] for yi in y_ids_train]

In [30]:
y_group_test = [specialty_to_group[str(yi)] for yi in y_ids_test]

In [73]:
feature_path = 'data/count_vectorizer_vocab.pkl'
with open(feature_path, 'wb') as fw:
     pickle.dump(cv.vocabulary_, fw)

In [74]:
len(cv.vocabulary_)

512

### Clasifier

In [31]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

In [63]:
clf.fit(X_train, y_group_train)

MultinomialNB()

In [64]:
X_test

<5100x512 sparse matrix of type '<class 'numpy.int64'>'
	with 15263 stored elements in Compressed Sparse Row format>

In [65]:
y_test_pred = clf.predict(X_test)

In [66]:
y_train_pred = clf.predict(X_train)

In [67]:
from sklearn.metrics import classification_report

In [68]:
print(classification_report(y_true=y_group_test, y_pred=y_test_pred))

              precision    recall  f1-score   support

           1       0.51      0.28      0.37       194
           4       0.57      0.07      0.12        58
           5       0.00      0.00      0.00        34
           7       0.31      0.21      0.25       399
           8       0.50      0.22      0.31       250
          10       0.00      0.00      0.00         2
          11       0.00      0.00      0.00        37
          13       0.26      0.80      0.40       739
          14       0.00      0.00      0.00         8
          15       0.42      0.47      0.44       381
          16       0.58      0.21      0.30        34
          17       0.00      0.00      0.00        14
          18       0.12      0.02      0.03        52
          20       0.59      0.25      0.35        92
          21       0.33      0.03      0.05        38
          22       0.00      0.00      0.00        13
          23       0.56      0.42      0.48       326
          25       0.75    

/home/dbabenko/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [69]:
import pickle

In [70]:
pickle.dump(clf, open("./data/multinomialNB.sav", 'wb'))

### Random forest 

In [38]:
from sklearn.ensemble import RandomForestClassifier

In [51]:
rf_cls = RandomForestClassifier(n_estimators=1000)

In [52]:
rf_cls.fit(X_train, y_group_train)

RandomForestClassifier(n_estimators=1000)

In [53]:
y_test_pred = rf_cls.predict(X_test)

In [54]:
print(classification_report(y_true=y_group_test, y_pred=y_test_pred))

              precision    recall  f1-score   support

           1       0.37      0.29      0.33       194
           4       0.35      0.10      0.16        58
           5       0.15      0.06      0.09        34
           7       0.28      0.24      0.25       399
           8       0.39      0.24      0.29       250
          10       0.00      0.00      0.00         2
          11       0.06      0.03      0.04        37
          13       0.29      0.63      0.40       739
          14       0.00      0.00      0.00         8
          15       0.36      0.39      0.37       381
          16       0.43      0.26      0.33        34
          17       0.00      0.00      0.00        14
          18       0.07      0.04      0.05        52
          20       0.39      0.27      0.32        92
          21       0.31      0.32      0.31        38
          22       0.00      0.00      0.00        13
          23       0.47      0.44      0.45       326
          25       0.29    

/home/dbabenko/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [61]:
np.unique(y_group_test).shape

(52,)

In [20]:
vectorizer = CountVectorizer(tokenizer=get_important_tokens,preprocessor=None, max_features=512, decode_error='ignore')

In [21]:
vectorizer.fit_transform(["Красное высыпание на боку живота"]).toarray()

array([[1, 1, 1, 1]])

In [22]:
df_train['comment'].values

array(['Красное высыпание на боку живота',
       'періодична сипь на руці. по ознаках схожа на крапивницю',
       'Номер направлення 6303-9261-6693-4577', ...,
       'Сломан тазобедренный сустав 10дней назад,одет гипс. Гипс начал расползаться и причинить боль. Нужна консультация. Тете 74года и весит она более 120 кг. Не стает с кровати, только сидит',
       'По времени желателен промежуток между 9:00 - 13:00.\nПо ближайшим дням - среда, пятница',
       '7,5 месяцев, пока не ползает и не сидит, в 6 месяцев начала активно переворачиваться со спины на живот. Когда делаю зарядку то чувствую сильное напряжение в ногах и сопротивление.'],
      dtype=object)

In [ ]:
X_train = vectorizer.fit_transform(df_train['comment'].values).toarray()

,id,comment,specialty_id
0,19600,Красное высыпание на боку живота,15
1,172287,періодична сипь на руці. по ознаках схожа на к...,15
2,1078909,Номер направлення 6303-9261-6693-4577,8
3,45659,"Удаление двух родинок на спине, скорее всего х...",81
4,4647,нужна консультация,53
...,...,...,...
20955,784405,"Здравствуйте, хочу проконсультироваться на счё...",38
20956,80062,гінеколог,13
20957,13016,"Сломан тазобедренный сустав 10дней назад,одет ...",34
20958,330832,По времени желателен промежуток между 9:00 - 1...,8


In [54]:
df_train['comment'].replace('', np.nan, inplace=True)

In [55]:
df_train.dropna()

,id,comment,specialty_id
0,19600,Красное высыпание на боку живота,15
1,172287,періодична сипь на руці. по ознаках схожа на к...,15
2,1078909,Номер направлення 6303-9261-6693-4577,8
3,45659,"Удаление двух родинок на спине, скорее всего х...",81
4,4647,нужна консультация,53
...,...,...,...
20955,784405,"Здравствуйте, хочу проконсультироваться на счё...",38
20956,80062,гінеколог,13
20957,13016,"Сломан тазобедренный сустав 10дней назад,одет ...",34
20958,330832,По времени желателен промежуток между 9:00 - 1...,8


In [57]:
df_train['comment'].values.shape

(20960,)

In [58]:
df_train['comment'].values

array(['Красное высыпание на боку живота',
       'періодична сипь на руці. по ознаках схожа на крапивницю',
       'Номер направлення 6303-9261-6693-4577', ...,
       'Сломан тазобедренный сустав 10дней назад,одет гипс. Гипс начал расползаться и причинить боль. Нужна консультация. Тете 74года и весит она более 120 кг. Не стает с кровати, только сидит',
       'По времени желателен промежуток между 9:00 - 13:00.\nПо ближайшим дням - среда, пятница',
       '7,5 месяцев, пока не ползает и не сидит, в 6 месяцев начала активно переворачиваться со спины на живот. Когда делаю зарядку то чувствую сильное напряжение в ногах и сопротивление.'],
      dtype=object)